### One-Time Data Setup

In [51]:
import pandas as pd
from pathlib import Path
from src.config import INTERIM_DATA_DIR, RAW_DATA_DIR
from sklearn.model_selection import train_test_split
import re

stats_path: Path = RAW_DATA_DIR / "stats" / "Stats-0.csv"
seasons_path: Path = RAW_DATA_DIR / "seasons" / "Season-0.csv"
metadata_path: Path = RAW_DATA_DIR / "metadata" / "Metadata-0.csv"
train_path: Path = INTERIM_DATA_DIR / "trainset.csv"
test_path: Path = INTERIM_DATA_DIR / "testset.csv"
base_path: Path = INTERIM_DATA_DIR / "base.csv",

In [52]:
def url_to_key(url):
    key = re.search('boxscores/(.+?).htm', url).group(1)
    return key

In [53]:
df_stats = pd.read_csv(str(stats_path).replace('0','2000'))

In [54]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                518 non-null    int64  
 1   event_date            518 non-null    object 
 2   nano                  518 non-null    object 
 3   market                518 non-null    object 
 4   name                  518 non-null    object 
 5   alias                 518 non-null    object 
 6   rush_att              518 non-null    int64  
 7   rush_yds              518 non-null    int64  
 8   rush_tds              518 non-null    int64  
 9   pass_cmp              518 non-null    int64  
 10  pass_att              518 non-null    int64  
 11  pass_cmp_pct          0 non-null      float64
 12  pass_yds              518 non-null    int64  
 13  pass_tds              518 non-null    int64  
 14  pass_int              518 non-null    int64  
 15  passer_rating         5

In [55]:
for i in range(2001, 2024):
    df_stats = pd.concat([df_stats, pd.read_csv(str(str(stats_path).replace('0',str(i))))], ignore_index=True)

In [56]:
df_stats['boxscore_stats_link'] = df_stats['boxscore_stats_link'].map(lambda url: url_to_key(url))

In [57]:
df_seasons = pd.read_csv(str(seasons_path).replace('0','2000'))

In [58]:
df_seasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   status               259 non-null    object
 1   season               259 non-null    int64 
 2   week                 259 non-null    int64 
 3   week_day             259 non-null    object
 4   event_date           259 non-null    object
 5   tm_nano              259 non-null    object
 6   tm_market            259 non-null    object
 7   tm_name              259 non-null    object
 8   tm_alias             259 non-null    object
 9   tm_alt_market        259 non-null    object
 10  tm_alt_alias         259 non-null    object
 11  opp_nano             259 non-null    object
 12  opp_market           259 non-null    object
 13  opp_name             259 non-null    object
 14  opp_alias            259 non-null    object
 15  opp_alt_market       259 non-null    object
 16  opp_alt_

In [59]:
for i in range(2001, 2024):
    df_seasons = pd.concat([df_seasons, pd.read_csv(str(str(seasons_path).replace('0',str(i))))], ignore_index=True)

In [60]:
df_seasons['boxscore_stats_link'] = df_seasons['boxscore_stats_link'].map(lambda url: url_to_key(url))

In [61]:
df_metadata = pd.read_csv(str(metadata_path).replace('0', '2000'))
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   season                      259 non-null    int64  
 1   event_date                  259 non-null    object 
 2   tm_nano                     259 non-null    object 
 3   tm_market                   259 non-null    object 
 4   tm_name                     259 non-null    object 
 5   tm_alias                    259 non-null    object 
 6   opp_nano                    259 non-null    object 
 7   opp_market                  259 non-null    object 
 8   opp_name                    259 non-null    object 
 9   opp_alias                   259 non-null    object 
 10  tm_spread                   259 non-null    float64
 11  opp_spread                  259 non-null    float64
 12  total                       259 non-null    float64
 13  attendance                  259 non

In [62]:
for i in range(2001, 2024):
    df_metadata = pd.concat([df_metadata, pd.read_csv(str(metadata_path).replace('0',str(i)))], ignore_index=True)

In [63]:
df_metadata['boxscore_stats_link'] = df_metadata['boxscore_stats_link'].map(lambda url: url_to_key(url))

In [64]:
df_metadata = df_metadata.rename(columns={"boxscore_stats_link":"id"}).set_index('id')
df_seasons = df_seasons.rename(columns={"boxscore_stats_link":"id"}).set_index('id')
df_stats = df_stats.rename(columns={"boxscore_stats_link":"id"}).set_index('id')

In [65]:
def standardize_seasons(r: pd.Series):
    if r.loc['tm_location'] == 'H':
        return r
    
    cols = ['nano', 'name', 'market', 'alias', 'alt_market', 'alt_alias', 'score', 'location']
    for col in cols:
        r.loc['tm_'+col] , r.loc['opp_'+col] = r.loc['opp_'+col], r.loc['tm_'+col]
    
    return r

df_seasons = df_seasons.apply(standardize_seasons, axis=1)

In [66]:
unique_seasons_cols = df_seasons.columns.difference(df_metadata.columns)
df = df_metadata.join(df_seasons[unique_seasons_cols], validate='one_to_one')

In [67]:
unique_stats_cols = df_stats.columns.difference(df.columns)
print(unique_stats_cols)
df = df.merge(df_stats[unique_stats_cols].add_prefix('tm_'), on=['id','tm_alias'], suffixes=('', '_y'), validate='one_to_one')
df = df.merge(df_stats[unique_stats_cols].add_prefix('opp_'), on=['id','opp_alias'], suffixes=('', '_y'), validate='one_to_one')

Index(['alias', 'first_downs', 'fourth_down_att', 'fourth_down_conv',
       'fourth_down_conv_pct', 'fumbles', 'fumbles_lost', 'market', 'name',
       'nano', 'net_pass_yds', 'pass_att', 'pass_cmp', 'pass_cmp_pct',
       'pass_int', 'pass_tds', 'pass_yds', 'passer_rating', 'penalties',
       'penalty_yds', 'rush_att', 'rush_tds', 'rush_yds', 'third_down_att',
       'third_down_conv', 'third_down_conv_pct', 'time_of_possession',
       'times_sacked', 'total_yds', 'turnovers', 'yds_sacked_for'],
      dtype='object')


In [68]:
df.drop(df.filter(regex='_y$').columns, axis=1, inplace=True)
df.columns

Index(['season', 'event_date', 'tm_nano', 'tm_market', 'tm_name', 'tm_alias',
       'opp_nano', 'opp_market', 'opp_name', 'opp_alias', 'tm_spread',
       'opp_spread', 'total', 'attendance', 'duration', 'roof_type',
       'surface_type', 'won_toss', 'won_toss_decision', 'won_toss_overtime',
       'won_toss_overtime_decision', 'temperature', 'humidity_pct',
       'wind_speed', 'game_time', 'opp_alt_alias', 'opp_alt_market',
       'opp_location', 'opp_score', 'status', 'tm_alt_alias', 'tm_alt_market',
       'tm_location', 'tm_score', 'week', 'week_day', 'tm_first_downs',
       'tm_fourth_down_att', 'tm_fourth_down_conv', 'tm_fourth_down_conv_pct',
       'tm_fumbles', 'tm_fumbles_lost', 'tm_net_pass_yds', 'tm_pass_att',
       'tm_pass_cmp', 'tm_pass_cmp_pct', 'tm_pass_int', 'tm_pass_tds',
       'tm_pass_yds', 'tm_passer_rating', 'tm_penalties', 'tm_penalty_yds',
       'tm_rush_att', 'tm_rush_tds', 'tm_rush_yds', 'tm_third_down_att',
       'tm_third_down_conv', 'tm_third_down_

In [69]:
'''df.to_csv(*base_path)

df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)
df_train.to_csv(str(train_path))
df_test.to_csv(str(test_path))'''

'df.to_csv(*base_path)\n\ndf_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)\ndf_train.to_csv(str(train_path))\ndf_test.to_csv(str(test_path))'

In [70]:
df_tm_names = df[['tm_name', 'tm_market', 'tm_alias']]
df_tm_names = df_tm_names.drop_duplicates()
def remove_prefix(prefix):
    return lambda x: x[len(prefix):]
df_tm_names = df_tm_names.rename(remove_prefix('tm_'), axis='columns').set_index('alias')

In [71]:
from src.config import DB_DATA_DIR
tm_names_path = DB_DATA_DIR / "tm_names.csv"
df_tm_names.to_csv(str(tm_names_path))